In [1]:
pip install ffmpeg-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import ffmpeg
import numpy as np
path = os.getcwd()
abspath = os.path.abspath(os.path.join(path, os.pardir))

inputPath = f'{abspath}/ffmpegTest/'
print(inputPath)

in_filename = inputPath + 'vid.mp4'
out_filename = inputPath + 'test.mp4'
overlay_filename = inputPath + 'box.png'


/tf/src/ffmpegTest/


In [11]:
process1 = (
    ffmpeg
    .input(in_filename)
    .output('pipe:', format='rawvideo', pix_fmt='rgb24', vframes=8)
    .run_async(pipe_stdout=True)
)

process2 = (
    ffmpeg
    .input('pipe:', format='rawvideo', pix_fmt='rgb24', s='{}x{}'.format(width, height))
    .output(out_filename, pix_fmt='yuv420p')
    .overwrite_output()
    .run_async(pipe_stdin=True)
)

while True:
    in_bytes = process1.stdout.read(width * height * 3)
    if not in_bytes:
        break
    in_frame = (
        np
        .frombuffer(in_bytes, np.uint8)
        .reshape([height, width, 3])
    )

    # See examples/tensorflow_stream.py:
    out_frame = in_frame

    process2.stdin.write(
        out_frame
        .astype(np.uint8)
        .tobytes()
    )

process2.stdin.close()
process1.wait()
process2.wait()

ValueError: cannot reshape array of size 1118400 into shape (1000,1000,3)

In [2]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import ffmpeg
import ipywidgets as widgets
import numpy as np

In [3]:
probe = ffmpeg.probe(in_filename)
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')
width = int(video_info['width'])
height = int(video_info['height'])
num_frames = int(video_info['nb_frames'])

In [14]:
out, err = (
    ffmpeg
    .input(in_filename)
    .output('pipe:', format='rawvideo', pix_fmt='rgb24')
    .run(capture_stdout=True)
)
video = (
    np
    .frombuffer(out, np.uint8)
    .reshape([-1, height, width, 3])
)

@interact(frame=(0, num_frames))
def show_frame(frame=0):
    plt.imshow(video[frame,:,:,:])

interactive(children=(IntSlider(value=0, description='frame', max=901), Output()), _dom_classes=('widget-inter…

In [4]:
from io import BytesIO
from PIL import Image


def extract_frame(stream, frame_num):
    while isinstance(stream, ffmpeg.nodes.OutputStream):
        stream = stream.node.incoming_edges[0].upstream_node.stream()
    out, _ = (
        stream
        .filter_('select', 'gte(n,{})'.format(frame_num))
        .output('pipe:', format='rawvideo', pix_fmt='rgb24', vframes=1)
        .run(capture_stdout=True, capture_stderr=True)
    )
    return np.frombuffer(out, np.uint8).reshape([height, width, 3])


def png_to_np(png_bytes):
    buffer = BytesIO(png_bytes)
    pil_image = Image.open(buffer)
    return np.array(pil_image)
    

def build_graph(
        enable_overlay, flip_overlay, enable_box, box_x, box_y,
        thickness, color):

    stream = ffmpeg.input(in_filename)

    if enable_overlay:
        overlay = ffmpeg.input(overlay_filename)
        if flip_overlay:
            overlay = overlay.hflip()
        stream = stream.overlay(overlay)

    if enable_box:
        stream = stream.drawbox(
            box_x, box_y, 120, 120, color=color, t=thickness)

    return stream.output(out_filename)


def show_image(ax, stream, frame_num):
    try:
        image = extract_frame(stream, frame_num)
        ax.imshow(image)
        ax.axis('off')
    except ffmpeg.Error as e:
        print(e.stderr.decode())


def show_graph(ax, stream, detail):
    data = ffmpeg.view(stream, detail=detail, pipe=True)
    image = png_to_np(data)
    ax.imshow(image, aspect='equal', interpolation='hanning')
    ax.set_xlim(0, 1100)
    ax.axis('off')


@interact(
    frame_num=(0, num_frames),
    box_x=(0, 200),
    box_y=(0, 200),
    thickness=(1, 40),
    color=['red', 'green', 'magenta', 'blue'],
)
def f(
        enable_overlay=True,
        enable_box=True,
        flip_overlay=True,
        graph_detail=False,
        frame_num=0,
        box_x=50,
        box_y=50,
        thickness=5,
        color='red'):

    stream = build_graph(
        enable_overlay,
        flip_overlay,
        enable_box,
        box_x,
        box_y,
        thickness,
        color
    )

    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15,4))
    plt.tight_layout()
    show_image(ax0, stream, frame_num)
    show_graph(ax1, stream, graph_detail)

interactive(children=(Checkbox(value=True, description='enable_overlay'), Checkbox(value=True, description='en…

In [2]:
import os
import ffmpeg
import numpy as np
path = os.getcwd()
abspath = os.path.abspath(os.path.join(path, os.pardir))

inputPath = f'{abspath}/ffmpegTest/'
print(inputPath)

in_filename = inputPath + 'merkel_Trim.mp4'
out_filename = inputPath + 'test.mp4'
overlay_filename = inputPath + 'pic.png'

/tf/src/ffmpegTest/


In [3]:
import os
import ffmpeg

in_file = ffmpeg.input(in_filename)
border_box = ffmpeg.input(overlay_filename)

(
    ffmpeg.overlay(
        border_box, in_file, x=50, y=50
    )
    .output(out_filename)
    .run()
)

Error: ffmpeg error (see stderr output for detail)